# Decisiveness: The influence of whales in DAO Governance 

## Definition

**Decisiveness** is [defined](https://www.merriam-webster.com/dictionary/decisive) as having the power or quality of deciding as in *The council president cast the decisive vote.*

More precisely, decisiveness here is used in the [game-theoretical sense](https://www.jstor.org/stable/30026126) of *casting a vote that affects the outcome of the election*.

## Aims of this report

This article aims to analyze the top 60 DAOs, ranked by [treasury size](https://deepdao.io/) to determine the impact of large token holders (whales) on their governance.

Our main contribution is the **_whale decisiveness_ metric**, representing how governance decisions in a given DAO are affected by whale votes.

In this report we produce:

- The whale decisiveness metric on each of the top 60 DAOs
- On three proposals, a case study on the influence whales held over the outcome

For the purpose of this report, whales are defined as the **top 5% voters in terms of voting power** for a given proposal (voters with voting power at or above the 95th percentile of voting power for that proposal). 

The whale decisiveness metric is calculated for each DAO as the percentage of proposals, **the outcome of which change once whale votes are not counted**.

This report relies on on-chain data as well as Snapshot data about proposals and voters.  
For more detailed data and case studies, check out our [extensive report here](https://acemasterjb.github.io/gov_analysis/analysis/extensive_analysis).

This is a work in progress and will be updated as we learn more, so stay tuned!

## Takeaways

Out of 60 DAOs:

- 5 of them (8%) have a whale decisiveness metric > 20%
- 16 of them (27%) have whale decisiveness > 10%

In 16 DAOs, whales' votes produced a different outcome than minority votes in 10 proposals of the last 100.

Reviewing proposals (in Decentraland, Curve Finance and Radicle) in detail, we hint at the reasons why whale votes diverge from minority votes. Namely:

- Massive whale voting power dominance coupled with meager participation from minority holders
- Divergence of interests

## Calculations

Below, a brief summary of calculations performed to build the whale decisiveness statistics.

See the [repository](https://github.com/butterymoney/gov_analysis) for more details.

In [40]:
# sets up the pynb environment
import os
import sys

from IPython.display import HTML
import pandas as pd

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from stages.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
)

### Load data

Load each voter's choice and voting power for up to the last one hundred proposals in each DAO.

In [12]:
all_organization_proposals = pd.read_csv(
    "../plutocracy_data/full_report/plutocracy_report.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)
all_organization_proposals_filtered = pd.read_csv(
    "../plutocracy_data/full_report/plutocracy_report_filtered.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)


In [18]:
def to_organization_map(flat_organization_dataframe: pd.DataFrame):
    return {
        organization_name: proposal_df
        for organization_name, proposal_df in [
            (str(space_name), space_proposals)
            for space_name, space_proposals in flat_organization_dataframe.groupby(
                "proposal_space_name"
            )
        ]
    }

plutocracy_report_data = to_organization_map(all_organization_proposals)
plutocracy_report_data_filtered = to_organization_map(all_organization_proposals_filtered)


In [19]:
pd.set_option("display.max_rows", int(1e3))
score_differences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)


### Compute score differences

For each choice of each proposal, get:

- Actual score.
- Hypothetical score that would have been produced if whales didn't vote.

Then compute if the outcome is different, meaning if the outcome would have been changed if whales didn't vote.

Then compute, for each DAO, the changed outcome propotion among the last 100 proposals.

In [20]:
score_differences_dfs = dict()

initial_series_data = {
    organization: 0
    for organization in plutocracy_report_data.keys()
}
changed_outcome_proportions = pd.Series(initial_series_data, name="changed outcomes %")

for score_difference in score_differences:
    for organization, data in score_difference.items():
        data: dict[str, list] = data
        items = data.items()
        score_differences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([proposal_id for proposal_id, _ in items]), name="Proposal ID"
            ),
            columns=[
                "score_differences",
                "whale_vp_proportion",
                "total_vp",
                "outcome_changed",
                "outcome_old",
                "outcome_new"
            ],
        ).astype({"total_vp": "float64"}, copy=False
        ).sort_values(["whale_vp_proportion","total_vp"], ascending=False)

        try:
            changed_outcome_proportions[organization] = score_differences_dfs[organization]["outcome_changed"].value_counts(normalize=True)[True]
        except KeyError:
            changed_outcome_proportions[organization] = 0

        space_id = plutocracy_report_data[organization].iloc[0]["proposal_space_id"]

        score_differences_dfs[organization]["total_vp"] = score_differences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)

        score_differences_dfs[organization].index = score_differences_dfs[organization].index.to_series().apply(
            lambda s: f'<a href=http://snapshot.org/#/{space_id}/proposal/{s} rel="noopener noreferrer" target="_blank">{s[0:9]}</a>'
        )
        score_differences_dfs[organization].style.format({"whale_vp_proportion": "{:.2%}".format})

changed_outcome_proportions_raw = changed_outcome_proportions.copy()
changed_outcome_proportions = changed_outcome_proportions.apply(
    lambda proportion: "{:.0%}".format(proportion)
)

In [21]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)

### Synthesis

For each DAO, show the percentage of proposals, the outcome of which would have changed if whales didn't vote (_whale decisiveness_).

In [22]:
dao_overview = pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
    ],
)
dao_overview.set_index(
    pd.Index([list(result.items())[0][0] for result in voting_ratios], name="DAO"),
    inplace=True
)

dao_overview.insert(2, "whale decisiveness", changed_outcome_proportions)
dao_overview

,# of whales,all voters,whale decisiveness
DAO,,,
1inch Network,26,604,0%
Aavegotchi,47,5624,7%
Alchemix,49,2212,4%
Ampleforth,6,119,6%
Angle Protocol,17,475,10%
ApeCoin DAO,192,6812,12%
BadgerDAO,251,11169,5%
Cryptex,4,24,6%
Curve Finance,21,285,21%


## Case studies

### Decentraland

#### Proportion of Outcomes Changed:

In [8]:
print(f"{changed_outcome_proportions['Decentraland']} of Decentraland's proposal outcomes change after filtering out whale voting power.")

27% of Decentraland's proposal outcomes change after filtering out whale voting power.


#### Proposal Analysis:

For example, [this proposal](https://snapshot.org/#/snapshot.dcl.eth/proposal/0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5) to set a duration period for the tenure of Decentraland DAO committee members had decisive whale votes; had whales not voted, the outcome would have been inversed.

In [9]:
propsal_choices = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Decentraland"].index.to_series().apply(
    lambda s: "0x7f6fed8" in s,
)
proposal_score_differences = score_differences_dfs["Decentraland"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_scores']

non_whales = [
    x - y for x, y in zip(eval(proposal_scores), proposal_score_differences)
]

pd.DataFrame(
    {choice: [score, score_diff, non_whale_score] for choice, score, score_diff, non_whale_score in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences, non_whales)},
    index=["Scores", "Whale-only scores", "Non-whale scores"],
)

,Yes,No,Invalid question/options
Scores,168338.243827,1.394610e+06,0.025206
Whale-only scores,166209.638630,1.392896e+06,0.000000
Non-whale scores,2128.605197,1.714368e+03,0.025206


99.9% of voting power was attributed to whales, with 94.85% of proposal voting power allocated to voting for the proposal not to pass.

We also observe that non-whale voting power is very low in this proposal.

### Curve Finance

#### Proportion of Outcomes Changed:

In [10]:
print(f"{changed_outcome_proportions['Curve Finance']} of Curve Finance's proposal outcomes change after filtering out whale voting power.")

21% of Curve Finance's proposal outcomes change after filtering out whale voting power.


#### Proposal Analysis:

Another proposal which would have passed if not for whale intervention is this proposal to [add the XSTUSD-3CRV pair](https://snapshot.org/#/curve.eth/proposal/0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5) to Curve's [gauge](https://resources.curve.fi/reward-gauges/understanding-gauges) [controller](https://curve.readthedocs.io/dao-gauges.html#the-gauge-controller) to accrue CRV for liquidity providers of XSTUSD-3CRV. XSTUSD is a stablecoin deployed on Polkadot and Kusama that is backed by a synthetic token called XOR ([Sora](https://sora.org/)'s native token).

The [governance discussion](https://gov.curve.fi/t/proposal-to-add-xstusd-3crv-to-the-gauge-controller/2998/15) about the vote shows reckless promotion of XST.

![](./res/curve_governance_shenanigans.png)

[Quite a](https://twitter.com/runekek/status/1478166276979793922) [few people](https://twitter.com/FreddieRaynolds/status/1463960623402913797) had their concerns about XST early on.

We looked at the first 16 accounts which showed really strong support for this proposal, and almost [every single](https://gov.curve.fi/u/meowtopia) [one was](https://gov.curve.fi/u/LiquidityKing) [created within](https://gov.curve.fi/u/Ryandotrrr) 2 days of the proposal's forum post. One can reasonably conclude that they were created for the sole purpose of "shilling".

In [11]:
propsal_choices = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Curve Finance"].index.to_series().apply(
    lambda s: "0x0eb23ea" in s,
)
proposal_score_differences = score_differences_dfs["Curve Finance"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_scores']

non_whales = [
    x - y for x, y in zip(eval(proposal_scores), proposal_score_differences)
]


pd.DataFrame(
    {choice: [score, score_diff, non_whale_score] for choice, score, score_diff, non_whale_score in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences, non_whales)},
    index=["Scores", "Whale-only scores", "Non-whale scores"],
)

,Yes,No
Scores,45352.592228,529201.282019
Whale-only scores,23462.994316,511330.472493
Non-whale scores,21889.597912,17870.809526


Just over 4% of voting power for this proposal was allocated by whales to vote "Yes" (which is just over half the total voting power allocated to the "Yes" choice for this proposal), whereas 89% of whale voting power was allocated to voting "No" (97% of total voting power for this choice).

Clearly, large token holders had an incentive not to let this proposal pass. It seems some such large CRV whales voted sensibly.

Just over 10% of whale voting power for this proposal was allocated to voting "Aye" on this proposal (27% of voting power allocated to the "Aye" choice came from whales). Whereas 45% of voting power from whales was allocated to the "Nay" choice (73% of whale voting power allocated to "Nay").

### Radicle

#### Proportion of Outcomes Changed:

In [12]:
print(f"{changed_outcome_proportions['Radicle']} of Radicle's proposal outcomes change after filtering out whale voting power.")

17% of Radicle's proposal outcomes change after filtering out whale voting power.


#### Proposal Analysis:

For Radicle, we look at this proposal to [distribute RAD remaining](https://snapshot.org/#/gov.radicle.eth/proposal/QmepPgXwo5q9GipZFKa32rnxaYoo3LrfRqduinftbU3L3S) following a Liquidity Bootsrapping (LBP) round conducted in February '21. This leftover RAD was proposed to be redistributed to participants of the LBP, i.e. people who bought RAD in this period from the Balancer LBP for RAD tokens.

In [13]:
propsal_choices = plutocracy_report_data['Radicle'][plutocracy_report_data['Radicle']['proposal_id'] == 'QmepPgXwo5q9GipZFKa32rnxaYoo3LrfRqduinftbU3L3S'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Radicle"].index.to_series().apply(
    lambda s: "QmepPgXwo" in s,
)
proposal_score_differences = score_differences_dfs["Radicle"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Radicle'][plutocracy_report_data['Radicle']['proposal_id'] == 'QmepPgXwo5q9GipZFKa32rnxaYoo3LrfRqduinftbU3L3S'].iloc[0]['proposal_scores']

non_whales = [
    x - y for x, y in zip(eval(proposal_scores), proposal_score_differences)
]


pd.DataFrame(
    {choice: [score, score_diff, non_whale_score] for choice, score, score_diff, non_whale_score in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences, non_whales)},
    index=["Scores", "Whale-only scores", "Non-whale scores"],
)

,Aye,Nay
Scores,78136.278442,123126.401817
Whale-only scores,20708.270722,90218.650933
Non-whale scores,57428.007720,32907.750885


Just over 10% of voting power for this proposal was allocated by whales to vote "Aye" on this proposal (27% of voting power allocated to the "Aye" choice came from whales). Whereas 45% of voting power that came from whales was allocated to the "Nay" choice (73% of whale voting power allocated to "Nay").